In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func,inspect

In [2]:
df = pd.read_csv("blackfridat_rename.csv")

In [3]:
df.insert(0, 'new_id', range(1, 1+ len(df)))
df

,new_id,user_id,product_id,gender,age,occupation,city_category,stay_in_current_city_years,marital_status,product_category_1,product_category_2,product_category_3,purchase
0,1,1004023,P00000142,F,36-45,9,C,1,1,3,4.0,5.0,8390
1,2,1000876,P00114342,M,26-35,4,B,1,0,1,8.0,17.0,15496
2,3,1005365,P00053842,M,18-25,12,C,0,0,4,5.0,12.0,2138
3,4,1000285,P00108542,F,36-45,0,A,1,0,5,8.0,14.0,8663
4,5,1001373,P00196042,F,26-35,1,A,1,0,4,9.0,15.0,2846
5,6,1003075,P00057742,F,26-35,11,C,2,1,2,8.0,18.0,13318
6,7,1003641,P00046742,M,26-35,0,B,1,0,1,2.0,15.0,15455
7,8,1004004,P00063342,M,51-55,0,C,1,1,1,2.0,9.0,15628
8,9,1000245,P00066542,M,36-45,16,B,0,0,1,15.0,16.0,4031
9,10,1003399,P00243942,M,26-35,4,C,1,0,1,2.0,15.0,11979


In [4]:
# Create Engine
engine = create_engine("sqlite:///blackfriday.sqlite")

In [5]:
# df.set_index('new_id',inplace = False)

In [6]:
df

,new_id,user_id,product_id,gender,age,occupation,city_category,stay_in_current_city_years,marital_status,product_category_1,product_category_2,product_category_3,purchase
0,1,1004023,P00000142,F,36-45,9,C,1,1,3,4.0,5.0,8390
1,2,1000876,P00114342,M,26-35,4,B,1,0,1,8.0,17.0,15496
2,3,1005365,P00053842,M,18-25,12,C,0,0,4,5.0,12.0,2138
3,4,1000285,P00108542,F,36-45,0,A,1,0,5,8.0,14.0,8663
4,5,1001373,P00196042,F,26-35,1,A,1,0,4,9.0,15.0,2846
5,6,1003075,P00057742,F,26-35,11,C,2,1,2,8.0,18.0,13318
6,7,1003641,P00046742,M,26-35,0,B,1,0,1,2.0,15.0,15455
7,8,1004004,P00063342,M,51-55,0,C,1,1,1,2.0,9.0,15628
8,9,1000245,P00066542,M,36-45,16,B,0,0,1,15.0,16.0,4031
9,10,1003399,P00243942,M,26-35,4,C,1,0,1,2.0,15.0,11979


In [7]:
# Declare the base
Base = declarative_base()

In [8]:
df.columns

Index(['new_id', 'user_id', 'product_id', 'gender', 'age', 'occupation',
       'city_category', 'stay_in_current_city_years', 'marital_status',
       'product_category_1', 'product_category_2', 'product_category_3',
       'purchase'],
      dtype='object')

In [9]:
# pd.to_numeric(df.stay_in_current_city_years)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147936 entries, 0 to 147935
Data columns (total 13 columns):
new_id                        147936 non-null int32
user_id                       147936 non-null int64
product_id                    147936 non-null object
gender                        147936 non-null object
age                           147936 non-null object
occupation                    147936 non-null int64
city_category                 147936 non-null object
stay_in_current_city_years    147936 non-null object
marital_status                147936 non-null int64
product_category_1            147936 non-null int64
product_category_2            147936 non-null float64
product_category_3            147936 non-null float64
purchase                      147936 non-null int64
dtypes: float64(2), int32(1), int64(5), object(5)
memory usage: 14.1+ MB


In [11]:
# df.stay_in_current_city_years

In [12]:
# Define ORM 
# Define the ORM class for `BlackFriday`
### BEGIN SOLUTION
class Blackfriday(Base):
    
    __tablename__ = 'blackfriday'
    new_id = Column(Integer, primary_key=True)
    user_id = Column(Integer)
    product_id = Column(Text)
    gender = Column(Text)
    age = Column(Text)
    occupation = Column(Integer)
    city_category = Column(Text)
    stay_in_current_city_years = Column(Text)
    marital_status = Column(Integer)
    product_category_1 = Column(Integer)
    product_category_2 = Column(Float)
    product_category_3 = Column(Float)
    purchase = Column(Integer)
    
    def __repr__(self):
        return f"id={self.new_id}"

In [13]:
# Use Create all to crate the table 
Base.metadata.create_all(engine)

In [14]:
# Verify the table name exists in db
engine.table_names()

['blackfriday']

In [15]:
conn = engine.connect()

In [16]:
# load csv 
csv_df = df

In [17]:
# orient records does creates a list of data to write
data = csv_df.to_dict(orient='records')

In [18]:
data

[{'new_id': 1,
  'user_id': 1004023,
  'product_id': 'P00000142',
  'gender': 'F',
  'age': '36-45',
  'occupation': 9,
  'city_category': 'C',
  'stay_in_current_city_years': '1',
  'marital_status': 1,
  'product_category_1': 3,
  'product_category_2': 4.0,
  'product_category_3': 5.0,
  'purchase': 8390},
 {'new_id': 2,
  'user_id': 1000876,
  'product_id': 'P00114342',
  'gender': 'M',
  'age': '26-35',
  'occupation': 4,
  'city_category': 'B',
  'stay_in_current_city_years': '1',
  'marital_status': 0,
  'product_category_1': 1,
  'product_category_2': 8.0,
  'product_category_3': 17.0,
  'purchase': 15496},
 {'new_id': 3,
  'user_id': 1005365,
  'product_id': 'P00053842',
  'gender': 'M',
  'age': '18-25',
  'occupation': 12,
  'city_category': 'C',
  'stay_in_current_city_years': '0',
  'marital_status': 0,
  'product_category_1': 4,
  'product_category_2': 5.0,
  'product_category_3': 12.0,
  'purchase': 2138},
 {'new_id': 4,
  'user_id': 1000285,
  'product_id': 'P00108542',


In [19]:
# delete all rows in the table -- Optional
#conn.execute(blackfriday.delete())

In [20]:
# delete all rows in the table -- Optional
conn.execute(Blackfriday.__table__.delete())

In [21]:
# insert df into db as bulk
conn.execute(Blackfriday.__table__.insert(),data)

In [22]:
# insert csv into db tabe
def populate_table(engine,table,csvfile):
    # connect
    conn = engine.connect()    
    

In [23]:
# Use a basic query to validate that the data was inserted correctly for table `measurement`
### BEGIN SOLUTION
engine.execute("SELECT * FROM blackfriday LIMIT 1").fetchall()
### END SOLUTION

[(1, 1004023, 'P00000142', 'F', '36-45', 9, 'C', '1', 1, 3, 4.0, 5.0, 8390)]